In [1]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import json
import requests

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_credentials() -> dict:
    credentials = {}
    with open('credentials.txt') as f:
        for line in f.readlines():
            try:
                key, value = line.split(": ")
                credentials[key] = value.rstrip(" \n")
            except ValueError:
                print('Add your email, username, and password in credentials file')
                exit(0)
    return credentials

class Twitterbot:
    def __init__(self, email, username, password):
        self.email = email
        self.username = username
        self.password = password
        chrome_options = Options()

        # Define the path to the ChromeDriver
        self.service = Service(ChromeDriverManager().install())

        # Initialize the WebDriver with the Service
        self.bot = webdriver.Chrome(service=self.service, options=chrome_options)

    def login(self):
        bot = self.bot
        bot.get('https://twitter.com/i/flow/login')
        time.sleep(3)
        try:
            # Wait for the email input field to be present and enter the email
            email_input = WebDriverWait(bot, 10).until(
                EC.presence_of_element_located((By.NAME, 'text'))
            )
            email_input.send_keys(self.email)
            email_input.send_keys(Keys.RETURN)
            time.sleep(2)

            # Check if username input is requested
            try:
                username_input = WebDriverWait(bot, 5).until(
                    EC.presence_of_element_located((By.NAME, 'text'))
                )
                username_input.send_keys(self.username)
                username_input.send_keys(Keys.RETURN)
                time.sleep(2)
            except TimeoutException:
                # Username input not requested, proceed to password
                pass

            # Wait for the password input field to be present and enter the password
            password_input = WebDriverWait(bot, 10).until(
                EC.presence_of_element_located((By.NAME, 'password'))
            )
            password_input.send_keys(self.password)
            password_input.send_keys(Keys.RETURN)
            time.sleep(2)
        except (NoSuchElementException, TimeoutException) as e:
            print(f"An error occurred during login: {e}")

    def search_and_collect_tweets(self, search_query):
        bot = self.bot
        bot.get(f'https://twitter.com/search?q={search_query}&src=typed_query')
        time.sleep(3)
        tweet_text_elements = []

        # Scroll multiple times to load more tweets
        for _ in range(200):  # Further increased scroll times for more data
            bot.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            time.sleep(2)

            html_content = bot.page_source
            soup = BeautifulSoup(html_content, 'html.parser')

            # Find all elements with 'tweetText' and tweet dates
            tweet_text_elements.extend(soup.find_all('div', {'data-testid': 'tweetText'}))

        # Collect tweet data
        tweet_data = []
        for tweet_text_element in tweet_text_elements:
            tweet_text = tweet_text_element.text.strip().replace("\n", "")
            tweet_date_element = tweet_text_element.find_parent('article').find('time')
            tweet_date = tweet_date_element['datetime'] if tweet_date_element else 'No Date'
            tweet_data.append({'text': tweet_text, 'date': tweet_date})

        # Convert the tweet data to a DataFrame
        df = pd.DataFrame(tweet_data)
        return df

if __name__ == "__main__":
    credentials = get_credentials()
    bot = Twitterbot(credentials['email'], credentials['username'], credentials['password'])
    bot.login()

    search_queries = [
        "485Visa Australia",
        "StudentVisa500 Australia",
        "AustraliaImmigration",
        "AustraliaVisa",
        "TemporaryGraduate Australia",
        "Migration Australia",
        "ImmigrationPolicy Australia",
        "VisaUpdate Australia",
        "VisaApplication Australia",
        "StudyInAustralia",
        "Australia student visa",
        "Australia migration news",
        "Australia visa updates",
        "485 visa changes",
        "Australia immigration policy",
        "international students Australia",
        "post study work visa Australia",
        "Australian visa news",
        "temporary graduate visa",
        "Australia visa applications"
    ]

    # Initialize an empty list to collect DataFrames
    all_tweets = []

    # Collect DataFrames in the list
    for query in search_queries:
        tweet_df = bot.search_and_collect_tweets(query)
        all_tweets.append(tweet_df)

    # Concatenate all DataFrames into one DataFrame
    all_tweets_df = pd.concat(all_tweets, ignore_index=True)

    # Drop duplicate rows
    all_tweets_df = all_tweets_df.drop_duplicates()

    # Save the concatenated DataFrame to a single CSV file
    all_tweets_df.to_csv("twitter_df.csv", index=False)



In [ ]:
# Base URL for the Guardian API
baseUrl = "https://content.guardianapis.com/search?q="

def getdata(searchString, content, page, guardian_api_key):
    url = baseUrl + searchString + '&from-date=' + fromDate + '&to-date=' + toDate + '&api-key=' + guardian_api_key + '&page-size=' + '50' + '&page=' + page + '&production-office=aus'
    response = requests.get(url)
    if response.status_code != 200:
        return [], []
    data = json.loads(response.content)
    if 'response' in data and 'results' in data['response']:
        results = data['response']['results']
        contents = [result[content] for result in results if content in result]
        dates = [result['webPublicationDate'] for result in results if 'webPublicationDate' in result]
        return contents, dates
    else:
        return [], []

# Function to get credentials from the credentials.txt file
def get_credentials() -> dict:
    credentials = {}
    with open('credentials.txt') as f:
        for line in f.readlines():
            key, value = line.split(": ")
            credentials[key.strip()] = value.strip().strip('"')
    return credentials

# Fetch credentials
credentials = get_credentials()
api_key_guardian = credentials.get('guardian_api_key')

# Define the date range for the search results
fromDate = "2020-01-01"
toDate = datetime.now().strftime('%Y-%m-%d')

# Initialize lists to store article titles and publication dates
article_titles = []
publication_dates = []

# Define the search terms and the number of pages to fetch for each term
search_terms = [
        "485Visa",
        "StudentVisa500",
        "Immigration",
        "Visa",
        "TemporaryGraduate",
        "Migration",
        "ImmigrationPolicy",
        "VisaUpdate",
        "VisaApplication",
        "StudyInAustralia",
        "Australia student visa",
        "Australia migration news",
        "Australia visa updates",
        "485 visa changes",
        "Australia immigration policy",
        "international students Australia",
        "post study work visa Australia",
        "Australian visa news",
        "temporary graduate visa",
        "Australia visa applications"
]
pages_to_fetch = 10

# Fetch data for each search term and each page
for term in search_terms:
    for i in range(1, pages_to_fetch + 1):
        titles, dates = getdata(term, "webTitle", str(i), api_key)
        article_titles += titles
        publication_dates += dates

# Create a DataFrame from the collected data
df1 = pd.DataFrame({
    "Title": article_titles,
    "Publication Date": publication_dates
})

df1 = df1.drop_duplicates()


In [ ]:
# Define a function to check if a title contains relevant topics
def contains_relevant_topic(title, search_terms):
    for topic in search_terms:
        if topic.lower() in title.lower():
            return True
    return False

# Create a new DataFrame to store relevant articles
relevant_articles = []

# Iterate through each row in the DataFrame
for index, row in df1.iterrows():
    title = row['Title']
    # Check if the title contains a relevant topic and does not only contain hashtags
    if contains_relevant_topic(title, search_terms) and not title.startswith("#"):
        relevant_articles.append(row)

# Convert the list of relevant articles back to a DataFrame
filtered_df1 = pd.DataFrame(relevant_articles)

# Display the filtered DataFrame
filtered_df1.to_csv('guardian_data.csv')


In [ ]:
# # config
import random

chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(options=chrome_options)

search_queries = [
    "Visa 485 Australia",
    "Student Visa 500 Australia",
    "Australia Immigration",
    "Australia Visa",
    "Temporary Graduate Australia",
    "Migration Australia",
    "Immigration Policy Australia",
    "Visa Update Australia",
    "Visa Application Australia",
    "StudyInAustralia",
    "Australia student visa",
    "Australia migration news",
    "Australia visa updates",
    "485 visa changes",
    "Australia immigration policy",
    "international students Australia",
    "post study work visa Australia",
    "Australian visa news",
    "temporary graduate visa",
    "Australia visa applications",
]


def scroll_down_page(driver):
    random_sleep = random.randint(5, 9)
    last_height = driver.execute_script("return document.body.scrollHeight")
    i = 1
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random_sleep)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        i += 1
        if i > 10:
            break


posts = []

for query in search_queries:
    driver.get(f"https://www.reddit.com/search/?q={query}")

    scroll_down_page(driver)

    time.sleep(10)

    # get all search results
    reddit_feed = driver.find_element(By.TAG_NAME, "reddit-feed")

    reddit_posts = reddit_feed.find_elements(
        By.XPATH, "//faceplate-tracker[@role='article']"
    )

    for post in reddit_posts:
        try:
            data = post.find_element(By.XPATH, ".//a[@data-testid='post-title']")
            author = post.get_attribute
            href = data.get_attribute("href")
            id = href.split("/")[-3]
            text_body = data.text
            time_created = post.find_element(By.TAG_NAME, "time").get_attribute(
                "datetime"
            )
            posts.append([id, href, text_body, time_created])
        except Exception as e:
            print(f"Error processing query: {query}")
            print(e)
            break

    print(f"Finished processing query: {query}")

df = pd.DataFrame(posts, columns=["post_id", "href", "text_body", "time_created"])
df.to_csv("./data/reddit_posts.csv")

time.sleep(10)

def click_load_more_comments(driver, max_loads=20):
    load_count = 0
    while load_count < max_loads:
        try:

            load_more_button = driver.find_element(
                By.XPATH, "//button[contains(., 'View more comments')]"
            )

            load_more_button.click()

            time.sleep(3)

            load_count += 1

        except Exception as e:
            print(f"Error loading more comments: {e}")
            break


comments = []
for post in posts:
    driver.get(post[1])

    print(f"Processing post: {post[0]}")

    click_load_more_comments(driver=driver, max_loads=20)

    comments_tree = driver.find_elements(By.TAG_NAME, "shreddit-comment-tree")

    reddit_comments = driver.find_elements(By.XPATH, "//shreddit-comment[@depth='0']")

    for comment in reddit_comments:
        try:
            post_id = post[0]
            user = comment.get_attribute("author")
            print(comment.get_attribute("postid"))
            text_body = comment.find_element(By.XPATH, ".//div[@slot='comment']").text
            time_created = comment.find_element(By.TAG_NAME, "time").get_attribute(
                "datetime"
            )
        except Exception as e:
            continue

        comments.append([post_id, user, text_body, time_created])

comment_df = pd.DataFrame(
    comments, columns=["post_id", "user", "text_body", "time_created"]
)
comment_df.to_csv("./data/reddit_comments.csv")